In [38]:
#pip install vnstock
from vnstock import *
import investpy
import yfinance as yf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re # Regex

import datetime as dt
from datetime import date
from datetime import datetime
from datetime import timedelta

from time import sleep
import random
import os
import csv
from xlwt import Workbook #xử lý với file excel .xls
import xlwt
import time
today = date.today()
str_today = date.today().strftime("%d-%m-%Y")
str_today_weekday =  date.today().strftime("%d-%m-%Y-%A")


from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

import gspread
import pygsheets
from google.oauth2.service_account import Credentials

import requests

# Lấy giá cổ phiếu

In [39]:
list_symbol = ['AAA', 'ACB', 'AGG', 'ANV', 'APH', 'ASM', 'BCG', 'BID', 'BMI', 'BMP', 'BVH', 'BWE', 'CII',
 'CTD', 'CTG', 'DBC', 'DCM', 'DGC', 'DGW', 'DHC', 'DIG', 'DPM', 'DXG', 'DXS', 'EIB', 'FPT', 'GAS', 'GEG',
 'GEX', 'GMD', 'GVR', 'HBC', 'HCM', 'HDB', 'HDG', 'HNG', 'HPG', 'HPX', 'HSG', 'HT1', 'IMP', 'ITA', 'KBC',
 'KDC', 'KDH', 'KOS', 'LPB', 'MBB', 'MSB', 'MSN', 'MWG', 'NKG', 'NLG', 'NT2', 'NVL', 'OCB', 'PAN', 'PC1',
 'PDR', 'PHR', 'PLX', 'PNJ', 'POW', 'PPC', 'PTB', 'PVD', 'PVT', 'REE', 'SAB', 'SAM', 'SBT', 'SCR', 'SCS',
 'SHB', 'SJS', 'SSB', 'SSI', 'STB', 'SZC', 'TCB', 'TCH', 'TMS', 'TPB', 'VCB', 'VCG', 'VCI', 'VGC', 'VHC',
 'VHM', 'VIB', 'VIC', 'VJC', 'VND', 'VNM', 'VPB', 'VPI', 'VRE', 'VSH', 'CMG', 'CRE',
 'FUEIP100','FUEKIV30','FUEDCMID','FUEKIVFS','FUESSV50',
 'E1VFVN30','FUESSVFL','FUEMAV30','FUESSV30','FUEVFVND','FUEVN100']
#list_symbol = ['AAA', 'ACB', 'AGG']

start = (date.today() + dt.timedelta(days=-1500)).strftime("%Y-%m-%d") #1500 ngày từ ngày hiện tại ~1025 ngày giao dịch ~3năm
end  = date.today().strftime("%Y-%m-%d")

df = pd.DataFrame(columns=list_symbol) 
symbol_false = []

change = pd.DataFrame(columns=['Symbol', 'Price', 'Max 3 Years', '1 day', 'Price 30days', 'Price 3years']) #Tạo dataframe để chứa Day Change

print(f'Loading {len(list_symbol)} stock (Get: Close, Volume, Open)')
STT=1
for symbol in list_symbol:
    print(f'------------- {symbol} ------------- Loading ------------- {STT}/{len(list_symbol)} -------------')
    STT+=1
    try:
        data_year = stock_historical_data(symbol=symbol, 
                                          start_date=start, #year-month-day
                                          end_date=end)  
        #data_year = investpy.get_stock_historical_data(stock= symbol, country= 'vietnam', from_date = start, to_date = end)  

        if data_year.empty:
            raise Exception
    except:
        print(symbol, ' no data')
        symbol_false.append(symbol)
        continue

        
    #Tính % Change Day
    HomTruoc = data_year['Close'].iloc[-2]
    HomNay = data_year['Close'].iloc[-1]
    chg = ((HomNay - HomTruoc)/HomTruoc)*100
    chg = "{:.2f}".format(chg)
    #Lưu % Day Change .txt
    if len(data_year)>1000:
        change = change.append(dict(zip(change.columns,[symbol, HomNay, data_year['Close'].max(), chg, 
                                        data_year['Close'].iloc[-30], data_year['Close'].iloc[0] ])), ignore_index=True)
    else:
        change = change.append(dict(zip(change.columns,[symbol, HomNay, 0, chg, 
                                        0, 0 ])), ignore_index=True)        
        
        
    #Create columns Close, Volumn, Open price
    df[symbol] = data_year['Close']
    df['Volume ' + symbol] = data_year['Volume']
    df['Open ' + symbol] = data_year['Open']
   
    #Calculate the RSI
    delta = df[symbol].diff(1)
    delta = delta.dropna()
    up = delta.copy()
    down = delta.copy()
    up[up<0] = 0
    down[down>0] = 0
    AVG_Gain = up.ewm(com=14).mean()
    AVG_Loss = abs(down.ewm(com=14).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    df['RSI ' + symbol]=RSI
    sleep(2)

df['Date'] = data_year['TradingDate']

print('Finish calculating of RSI')
# Remove symbols have no data
symbol_clear = list(set(list_symbol) - set(symbol_false))
df

Loading 111 stock (Get: Close, Volume, Open)
------------- AAA ------------- Loading ------------- 1/111 -------------
------------- ACB ------------- Loading ------------- 2/111 -------------
------------- AGG ------------- Loading ------------- 3/111 -------------
------------- ANV ------------- Loading ------------- 4/111 -------------
------------- APH ------------- Loading ------------- 5/111 -------------
------------- ASM ------------- Loading ------------- 6/111 -------------
------------- BCG ------------- Loading ------------- 7/111 -------------
------------- BID ------------- Loading ------------- 8/111 -------------
------------- BMI ------------- Loading ------------- 9/111 -------------
------------- BMP ------------- Loading ------------- 10/111 -------------
------------- BVH ------------- Loading ------------- 11/111 -------------
------------- BWE ------------- Loading ------------- 12/111 -------------
------------- CII ------------- Loading ------------- 13/111 ---

C:\Users\wenlamsao\AppData\Local\Temp\ipykernel_9476\2525129836.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['RSI ' + symbol]=RSI


------------- FPT ------------- Loading ------------- 26/111 -------------


C:\Users\wenlamsao\AppData\Local\Temp\ipykernel_9476\2525129836.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Volume ' + symbol] = data_year['Volume']
C:\Users\wenlamsao\AppData\Local\Temp\ipykernel_9476\2525129836.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Open ' + symbol] = data_year['Open']
C:\Users\wenlamsao\AppData\Local\Temp\ipykernel_9476\2525129836.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

------------- GAS ------------- Loading ------------- 27/111 -------------


...

------------- FUEVN100 ------------- Loading ------------- 111/111 -------------


C:\Users\wenlamsao\AppData\Local\Temp\ipykernel_9476\2525129836.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Volume ' + symbol] = data_year['Volume']
C:\Users\wenlamsao\AppData\Local\Temp\ipykernel_9476\2525129836.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Open ' + symbol] = data_year['Open']
C:\Users\wenlamsao\AppData\Local\Temp\ipykernel_9476\2525129836.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

Finish calculating of RSI


C:\Users\wenlamsao\AppData\Local\Temp\ipykernel_9476\2525129836.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Date'] = data_year['TradingDate']


,AAA,ACB,AGG,ANV,APH,ASM,BCG,BID,BMI,BMP,...,Volume FUESSV30,Open FUESSV30,RSI FUESSV30,Volume FUEVFVND,Open FUEVFVND,RSI FUEVFVND,Volume FUEVN100,Open FUEVN100,RSI FUEVN100,Date
0,12635.0,11209.0,19361.0,24202.0,30167.0,5742.0,3233.0,24980.0,14797.0,39010.0,...,197330.0,11760.0,NaN,10.0,10500.0,NaN,2660.0,12540.0,NaN,2020-07-21
1,12321.0,11209.0,19202.0,23481.0,32226.0,5629.0,3149.0,25319.0,14797.0,38522.0,...,6540.0,10000.0,0.000000,10.0,11200.0,100.0,90820.0,13410.0,100.000000,2020-07-22
2,11968.0,11209.0,18788.0,23721.0,34468.0,5517.0,3092.0,25056.0,14730.0,38244.0,...,54180.0,9840.0,0.000000,10.0,11980.0,100.0,25130.0,14340.0,47.154472,2020-07-23
3,12164.0,11209.0,18883.0,23922.0,36527.0,5517.0,3098.0,25545.0,14395.0,38104.0,...,17700.0,9800.0,30.667878,873860.0,12810.0,100.0,2840.0,13370.0,65.713978,2020-07-24
4,11889.0,11133.0,19011.0,24002.0,34528.0,5477.0,3081.0,25582.0,13927.0,38244.0,...,104590.0,9910.0,58.162306,189470.0,12500.0,100.0,2640.0,12440.0,46.863579,2020-07-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,7120.0,NaN,NaN,29200.0,NaN,8750.0,6660.0,41800.0,24000.0,60600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1020,7160.0,NaN,NaN,29400.0,NaN,8720.0,6950.0,44700.0,23700.0,62000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1021,7400.0,NaN,NaN,30400.0,NaN,8930.0,7310.0,44950.0,24000.0,64500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1022,7580.0,NaN,NaN,31000.0,NaN,9550.0,7480.0,45100.0,24000.0,63500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [3]:
len(data_year)

629

### Change Days

In [40]:
change['Good Price'] = list(change['Price'] < change['Max 3 Years']/2)
change['30days'] = 0 and ((change['Price'] - change['Price 30days']) / change['Price 30days'])*100 
change['3years'] = 0 and  ((change['Price'] - change['Price 3years']) / change['Price 3years'])*100
change = change[['Symbol', 'Price', 'Max 3 Years', 'Good Price', '1 day', '30days', '3years']]
change

,Symbol,Price,Max 3 Years,Good Price,1 day,30days,3years
0,AAA,7590.0,22800.0,True,0.13,0,0
1,ACB,26000.0,30360.0,False,1.96,0,0
2,AGG,29850.0,0.0,False,0.00,0,0
3,ANV,31100.0,61560.0,False,0.32,0,0
4,APH,7360.0,0.0,False,0.27,0,0
...,...,...,...,...,...,...,...
106,FUESSVFL,16210.0,0.0,False,0.87,0,0
107,FUEMAV30,13280.0,0.0,False,0.99,0,0
108,FUESSV30,13800.0,0.0,False,-0.14,0,0
109,FUEVFVND,23800.0,0.0,False,-0.42,0,0


# RSI

In [52]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

first_cross = 0
df_RSI = pd.DataFrame(columns=['Symbol','Indicator','Date','Advice'])

#Loop throught the lenght of the data set
for symbol in symbol_clear:  
    for i in range(0, len(df)):
        if df['RSI ' + symbol][i] < 70 and df['RSI ' + symbol][i] > 50 and first_cross == 0:
            first_cross = 1
            df_RSI = df_RSI.append(dict(zip(df_RSI.columns,[symbol, 'RSI cross down 70 ', df.index[i], 'SELL'])), ignore_index=True)
            
        elif df['RSI ' + symbol][i] > 70 and first_cross == 1:
            first_cross = 0  
            df_RSI = df_RSI.append(dict(zip(df_RSI.columns,[symbol, 'RSI cross up 70 ', df.index[i], 'Wait to SELL'])), ignore_index=True)
                
        elif df['RSI ' + symbol][i] > 30 and df['RSI ' + symbol][i] < 50 and first_cross == 0:
            first_cross = 1                
            df_RSI = df_RSI.append(dict(zip(df_RSI.columns,[symbol, 'RSI cross up 30 ', df.index[i], 'BUY'])), ignore_index=True)
                
        elif df['RSI ' + symbol][i] < 30 and first_cross == 1:
            first_cross = 0               
            df_RSI = df_RSI.append(dict(zip(df_RSI.columns,[symbol, 'RSI cross down 30 ', df.index[i], 'Wait to BUY'])), ignore_index=True)

df_RSI['Date'] = df_RSI['Date'].astype(int)
list_RSI_duplicate_7days=list(set(df_RSI.loc[(df_RSI['Date']>1016) & (df_RSI['Advice']=='BUY')]['Symbol']))
print(list_RSI_duplicate_7days)

df_RSI

['VPI', 'HPX', 'CRE']


,Symbol,Indicator,Date,Advice
0,REE,RSI cross down 70,3,SELL
1,REE,RSI cross down 30,5,Wait to BUY
2,REE,RSI cross up 30,8,BUY
3,REE,RSI cross down 30,9,Wait to BUY
4,REE,RSI cross up 30,15,BUY
...,...,...,...,...
7136,SZC,RSI cross up 30,856,BUY
7137,SZC,RSI cross down 30,923,Wait to BUY
7138,SZC,RSI cross up 30,935,BUY
7139,SZC,RSI cross down 30,941,Wait to BUY


# Information

In [6]:
df_info = listing_companies()[['ticker','group_code','company_short_name']][listing_companies()['ticker'].isin(list_RSI_duplicate_7days)] # lọc theo danh sách list
df_info.reset_index(drop=True, inplace=True)
df_info

df_overview = pd.DataFrame(columns=['ticker','exchange','industry','establishedYear','website']) #tạo 1 DataFrame rỗng, có số columns là 1 list
for i in list_RSI_duplicate_7days:
    try:
        print(ticker_overview(i)['ticker'][0])
        df_overview = df_overview.append(dict(zip(df_overview.columns,[ticker_overview(i)['ticker'][0],
                                                                       ticker_overview(i)['exchange'][0],
                                                                       ticker_overview(i)['industry'][0],
                                                                       ticker_overview(i)['establishedYear'][0],
                                                                       ticker_overview(i)['website'][0]        ])), ignore_index=True) #đưa list thứ 1 vào DataFrame
    except:
        print(i, ' no data')
        #symbol_false.append(symbol)
        add_row_overview = {'ticker':i,
                            'exchange':0,
                            'industry':0,
                            'establishedYear':0,
                            'website':0}
        df_overview = df_overview.append(add_row_overview, ignore_index=True)
        continue
df_overview

df_merge = pd.merge(df_info, df_overview, left_on='ticker', right_on='ticker')
# =HYPERLINK("http://www.google.com/","Google")
df_merge['Link Chart'] = '=HYPERLINK("https://liveboard.cafef.vn/ptkt?symbol=' + df_merge['ticker'] + '","Link")'
df_merge

STB
HDB
VHC
CMG
BVH
FUEKIV30
VIB
VPI
VCB
PPC
VSH
GEX
KBC
TCB
NVL
DGC
LPB
FUEIP100
POW
FPT
HDG
BWE
ANV
MSN
DXS
FUEVN100
AGG
CTD
HNG
ITA
CRE
CII
BID
KDC
VND
TCH
VPB
VCI
FUESSV30
EIB
GEG
TPB
CTG
DIG
DXG


,ticker,group_code,company_short_name,exchange,industry,establishedYear,website,Link Chart
0,FUEIP100,VNINDEX,ETF IPAAM VN100,HOSE,Dịch vụ tài chính,2019,https://ipaam.com.vn/vi/home,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
1,FUEKIV30,VNINDEX,KIM GROWTH VN30 ETF,HOSE,Dịch vụ tài chính,2021,https://koreainvestment.com.vn/en/home,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
2,AGG,VNINDEX,Bất động sản An Gia,HOSE,Bất động sản,2012,https://angia.com.vn/,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
3,ANV,VNINDEX,Thủy sản Nam Việt,HOSE,Thực phẩm và đồ uống,2006,http://www.navicorp.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
4,BID,VNINDEX,BIDV,HOSE,Ngân hàng,1993,http://www.bidv.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
5,BWE,VNINDEX,Nước - Môi trường Bình Dương,HOSE,"Điện, nước & xăng dầu khí đốt",2006,http://www.biwase.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
6,BVH,VNINDEX,Tập đoàn Bảo Việt,HOSE,Bảo hiểm,2007,https://www.baoviet.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
7,CRE,VNINDEX,Bất động sản Thế Kỷ,HOSE,Bất động sản,2001,http://www.cenland.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
8,CII,VNINDEX,Hạ tầng Kỹ thuật TP.HCM,HOSE,Xây dựng và Vật liệu,2001,http://www.cii.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."
9,CMG,VNINDEX,Tập đoàn Công nghệ CMC,HOSE,Công nghệ Thông tin,2009,http://www.cmc.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy..."


# Tạo Chart và Save

In [30]:
for symbol in list_RSI_duplicate_7days:
    fig = plt.figure(figsize=(15,5))
    plt.fill_between(df.index, df[f"{symbol}"], df[f"{symbol}"].min(), alpha=0.7, color="green")

    fig.savefig(fr"E:\OneDrive\CNTT\Project Python\Python Finance\Chart\\{symbol}.jpg", bbox_inches='tight', dpi=150) #save lại Chart với dạng hình ảnh
    #fig.savefig(fr"E:\Chart\\{symbol}.jpg", bbox_inches='tight', dpi=150)
    plt.close()

# Link Drive

In [37]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from Google import Create_Service #thư viện Google này là file Google.py trong máy, không phải pip install(nếu lỡ cài thì pip uninstall google) 
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
import os
import io
import pandas as pd


CLIENT_SECRET_FILE = r'E:\OneDrive\CNTT\Project Python\Python Finance\OAuth_Desktop_Account.json'
#CLIENT_SECRET_FILE = r'E:\OAuth_Desktop_Account.json'
API_SERVICE_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']
service = Create_Service(CLIENT_SECRET_FILE, API_SERVICE_NAME, API_VERSION, SCOPES)

E:\OneDrive\CNTT\Project Python\Python Finance\OAuth_Desktop_Account.json-drive-v3-(['https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/drive']
drive service created successfully


# Up Chart lên Drive

In [9]:
'''
list_image = []
for i in list_symbol:
    i = str(i) + '.jpg'
    list_image.append(i)
print(list_image)

file_names = list_image
mime_types = ['image/jpeg']*len(list_image)

list_id = []
for file_name, mime_types in zip(file_names, mime_types):
    file_metadata = {'parents': ['1ppuCXFEZhMsO61LQSEQpgTwoJiijBu4L'], #folder chứa file
                    'name': file_name}

    media = MediaFileUpload(r'E:\OneDrive\CNTT\Project Python\Python Finance\Chart\{0}'.format(file_name), mimetype=mime_types)
    response = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    list_id.append(re.findall(r"{'id': '(.+)'}", (str(response)))[0])
list_id
'''

'\nlist_image = []\nfor i in list_symbol:\n    i = str(i) + \'.jpg\'\n    list_image.append(i)\nprint(list_image)\n\nfile_names = list_image\nmime_types = [\'image/jpeg\']*len(list_image)\n\nlist_id = []\nfor file_name, mime_types in zip(file_names, mime_types):\n    file_metadata = {\'parents\': [\'1ppuCXFEZhMsO61LQSEQpgTwoJiijBu4L\'], #folder chứa file\n                    \'name\': file_name}\n\n    media = MediaFileUpload(r\'E:\\OneDrive\\CNTT\\Project Python\\Python Finance\\Chart\\{0}\'.format(file_name), mimetype=mime_types)\n    response = service.files().create(body=file_metadata, media_body=media, fields=\'id\').execute()\n    list_id.append(re.findall(r"{\'id\': \'(.+)\'}", (str(response)))[0])\nlist_id\n'

In [10]:
list_id = ['1ur1A7nonZE3XnNQOJ_41tqcqLmVJMzf_',
 '1hjB99EB2voZz_J1ISYADmnh2JyIdrZRw',
 '1OBE7VAKNFNMWWL9M_aBiws1DnkQSJ1iq',
 '1KoSM2mz6y2jiT0ndFFEtYpR0z4cPXkD1',
 '141T4JORwiIyybSvmeT--j95rlehEZrzc',
 '1HaFnk1WxMHfOYLtJ5syMDkq8nQKiDTT0',
 '1HcKCLA9yW595WWtuY_dSw7CriVLWcf-s',
 '19Up94QCdCDvoJURylxYzQKEGECs81i0F',
 '1mbFk-8bUAa_7ngaTQR1WVD8xGqO4ABev',
 '1w6qMP8BMJ-m2K5X1NN0bXaP3n6Tzv18l',
 '1u-Q-VhUICrxxMxhQefqSSt6wGH1rxNhc',
 '1s6NaEHY51PBSOCtZ_O9YzS74YcEeG8JF',
 '1mJFGAMPQqexSixujcKBb7P9SDeRnsiVV',
 '1detq0WrOU8B9VBRShCOBTlyGg0w1jgE5',
 '16GcvOI1wJUEyuzCHAqNON84EiUSWw8MM',
 '14QxmAwzSKiXxpLn9urXiBrG14E8of_Ip',
 '1hcuO9i32kFxG8nx3oy9TOcP5KOA0a8TY',
 '1aWN01sVksIeHuPLd0duYcxeUngwLzuwj',
 '1cnsv9iMRuIa2-LDu0h-H3U4sEglw1wCe',
 '1XXURl78bl1KXdzjKFu8KZml6r1-xlpqC',
 '1kk2QlIvRPLwP6ipoufEjW3ZNpaO_PYbA',
 '1goDctWuxM0rpoLgnu8D7iWUL2B0loJ0g',
 '1M5zgmGzf_h8CMsEu12mNriVjcpYYjzu4',
 '1LRmkJtVk3PQ5qtKO8X3PE50KpMDCR3-t',
 '1OrwZH2aNZyiC0VN740-6tlq1nMoyMA-c',
 '1OEWgvrXeM7K7FFyA42SVafbAjC90hHMv',
 '1RJIQEjCQaMmb2Hau97_jJ08ptu6GCtsT',
 '1fE8alZrtm7A5jMD_3IbqHGXpa46clzWB',
 '1fAHfscAxmWm4rNfTzr2WMzGtzjstvjs-',
 '13LZ0CTif06nue4vKMLppq0K4Yf_hevNH',
 '1Yavvw0UMmUqnKFfB1Yv8HAFfAErJ_dRa',
 '1lIefWgaFb-QFVgEk083YQAA2Hl2_cgsh',
 '1VEEQ0ySWDOGL9XAOl_GwS-vzPbqhYwCx',
 '1pyEiEn6PznNK3YPIv2hIR2Pm_D780kVW',
 '1uRTp2v7AOtRWQXbh7X5YmNkXTGxEa8bQ',
 '1x5kwrYngdE--xJappfLcPoY43kagxEoe',
 '1XNJLBKHRuPlhP42ONXVMamPobJF26Al5',
 '1ld4yM2jB77cm73JvCApJSrFuMHQjDfID',
 '1GEW48QGUpgTmFx2_0WVZEvb54Kv01fl5',
 '1qQtjY4T2CCIndmo2O8rjF9xXgFzmMy55',
 '1V2pCVJj3Wf_ZVObTqKHu6BDnyjyofULp',
 '109g96Yc1aI2eR4YBhYENK9EGM3hzqV3y',
 '1cWR_VeDfLaXYYtwZ71pPuJ-jvdDmH03O',
 '151bucH6d5BwZr7vk-VqlirRhiRxjEpKt',
 '1z_QQdKE6R-X0WkyC7sG8R1AQ35yh-1Dq',
 '1zWd4pMD6sx6zv8tZzf05qKfoslnjSQT8',
 '18rqELs5QbddXvqV3q-TVJevWw7ppUO4o',
 '1MVP6Hks9xqLwCjWwt7OaY0Ud-NKPDloI',
 '1O6Lw1Ig7dsob8q-i7iJ-jUXrXYvPEIdA',
 '1AJLXsjKdU244jOz3UcXKUgY7RXnu2Q56',
 '1oBLPYQyYux3fUNU7z8lg0NB3ACPLU2Y_',
 '1dqLawMpu2ih5TvdEqekGldmUlCeMCqTu',
 '1g1KWps1u7XelNxdgzViK20cz8ATXqPSv',
 '1vieH3HpMn3UxIDoO3bLurRQgPbbIrPXI',
 '1PFZVtPDSlBYMKHNu353UVW0FaXLLRkfy',
 '1V4xSM3zMw0R2ScHkDUoJvGEPQ3BvIQgk',
 '1fomKLH-kTMvz5A1SLgPMoJAKiKXDYSx8',
 '1sSpMpxcqsY3XlFAqaaGFe45BWh9BIwWx',
 '1Y0nWV8J4CRNC9hqyNIYtEqlUBL5YviJS',
 '1H0iH4IcMyTyk9EM3ywjYFOAkcAlQi7A6',
 '1cZvzolRR5q5w85TYr117Q-IOBHBCAjgG',
 '1mEUzJaFcE2JKK6qjd_u5VURmI3o5iYyi',
 '1hLBiqT56dtRqMmGPLQj1lAoHAQXbOKQr',
 '1fqvjme9g3snTvZpt2ZZA98YOUa46wSQT',
 '1JOsSI9q50aVxj6Ca-_yzc5xdjWwXOT10',
 '1KLoBB7EuPgMoQJ6buVHICwkhTUx9A--1',
 '1r0HWTt5nWgDqdSZ3oFLUsp0uviVuGzYC',
 '10yNh-DrfaOXsYJx8tuC-1QzjQYuPKjso',
 '1q_Pts13XVzip_xfJO8NqFSaKcJnA0Hr6',
 '1JVwR8VmScGolnVnbGBlg34uSFImQxYXu',
 '16FuRLo6mIx0DY09leM2n9sdNrtnfM7Gz',
 '1Zk4BOtQ2LkFrZ348e7RchDw5Is_3m-hU',
 '1QL6v9zHuTb8SI3ytQWnApVR67I6poLHo',
 '1oBm4WKm-WU4omFkwARekigs1AcU8e56_',
 '1utWPf0_qI174zUNxl4GBa9nsVvBSyDqF',
 '1kKMPd-WZsfirsjgJKJ0zd08g4ooyvGdj',
 '1RHMO4UrUXHQ2k5FwdHsXnZ3t2i0WgoMu',
 '1C52mBvbHUNUx7zrGrxur3Hnvsi7vwFCK',
 '1jNd2bSB-KIxpsd_T18mLs8AR61QB6vK2',
 '1JGlYQmACl57dAJIRK_M-6NcRKLgtV3hX',
 '1BL9LJntTgiNX0Yq1yZyVY37uar3xwHzT',
 '1QAqcnb1yZbh_-138P4LOJDyyC5WiUbzb',
 '1qQHZNgsyPjbqYLNPUozdczxo1fJ4Mxpc',
 '1Ec3veR29-fQr_2BRQryRJRtAOOf1T5Te',
 '1AdH_35YLj-76xGH6YAPtIQk5bYdav86G',
 '1K_NoMuLtvJHn_RN2ILX9o6j0YeVZjvPb',
 '1n1qtSVIzbij4JZIur0s95hP_tkcbjxZe',
 '1znqgKuhSF1Q2ELF0Mi_d0eYwgcluOrds',
 '1YYxFidTDZBaHLliqdhR29H2pU_9rvuQz',
 '1sK7Hw5r0L35iYib54rbisMdXuAevwpDl',
 '1HM3DKeYfmDyfh3QxarG-J_sfc4QDEUiK',
 '1k4Q2Su6R-x94mmiw7pDQXoGD4jin3qcI',
 '1bD31_-Mni6v9ae794KzeAakKRBH7MysQ',
 '1O2K0CABGAjmbA6Pvy2rRLJEhp1mWrmPj',
 '1xU4GZ4xWU6pXJYaeLn7tMeKxyfF2jmqW',
 '10cC73JD2KYQG8-cj_OKmoXkvgYQScBIv',
 '1g17R8U1MExIThDumbhRhdWkzjevQvyH-',
 '1G3Lpua7uG16LZ5uAFhqXUmzaG2Jt9aqo',
 '1LXvXiCRNZgtO70O7Ewl4fONS9karjh0w',
 '1Q9KOaUlnWvezjFZBFhqtyL84MraRwoJE',
 '1fVtzCw4Wyg1k9VJuocSfPhsJ3-oC8SjH',
 '15x07ENc_DvTIg0yIW0s30ydbM1r63Gde',
 '1Pl9JrBmOfZYa2SFy-eg_xT2_eHw8V2e_',
 '1ql4oyZr-PcWJOq4ymhCihcTZ6OGh16Hg',
 '1T4kynv5ZGxSdFbkxykf79MZ7r5EKZRrZ',
 '1PFWI83B_DwLv2-hONvRTV0E-8u107JxH',
 '1Y94eyWWEZL1eC5WFCA6WUHxnxPPkNdHA',
 '1H3QRY1Tv1yM0Y1tRZEoWG4GPk5NNmsbR',
 '1-r1Ahxmy7O-uAYVZ9bBM475ktj8lWSVl',
 '1ZwDTra5Yekur-x1Lc0KVZEOHOV4ytDrt',
 '1pMLUSHzgTO1HAs6TR6pgOQP7O_A1sCSL']

df_id = pd.DataFrame({'ticker': list_symbol,
                      'id': list_id})
df_id

,ticker,id
0,AAA,1ur1A7nonZE3XnNQOJ_41tqcqLmVJMzf_
1,ACB,1hjB99EB2voZz_J1ISYADmnh2JyIdrZRw
2,AGG,1OBE7VAKNFNMWWL9M_aBiws1DnkQSJ1iq
3,ANV,1KoSM2mz6y2jiT0ndFFEtYpR0z4cPXkD1
4,APH,141T4JORwiIyybSvmeT--j95rlehEZrzc
...,...,...
106,FUESSVFL,1Y94eyWWEZL1eC5WFCA6WUHxnxPPkNdHA
107,FUEMAV30,1H3QRY1Tv1yM0Y1tRZEoWG4GPk5NNmsbR
108,FUESSV30,1-r1Ahxmy7O-uAYVZ9bBM475ktj8lWSVl
109,FUEVFVND,1ZwDTra5Yekur-x1Lc0KVZEOHOV4ytDrt


In [11]:
df_merge = pd.merge(df_merge, df_id, left_on='ticker', right_on='ticker')

str_3YearsChart = '=image("https://drive.google.com/uc?export=download&id=' + df_merge['id'] + '")'
df_merge['3 years chart'] = str_3YearsChart
df_merge

,ticker,group_code,company_short_name,exchange,industry,establishedYear,website,Link Chart,id,3 years chart
0,FUEIP100,VNINDEX,ETF IPAAM VN100,HOSE,Dịch vụ tài chính,2019,https://ipaam.com.vn/vi/home,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",1fVtzCw4Wyg1k9VJuocSfPhsJ3-oC8SjH,"=image(""https://drive.google.com/uc?export=dow..."
1,FUEKIV30,VNINDEX,KIM GROWTH VN30 ETF,HOSE,Dịch vụ tài chính,2021,https://koreainvestment.com.vn/en/home,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",15x07ENc_DvTIg0yIW0s30ydbM1r63Gde,"=image(""https://drive.google.com/uc?export=dow..."
2,AGG,VNINDEX,Bất động sản An Gia,HOSE,Bất động sản,2012,https://angia.com.vn/,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",1OBE7VAKNFNMWWL9M_aBiws1DnkQSJ1iq,"=image(""https://drive.google.com/uc?export=dow..."
3,ANV,VNINDEX,Thủy sản Nam Việt,HOSE,Thực phẩm và đồ uống,2006,http://www.navicorp.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",1KoSM2mz6y2jiT0ndFFEtYpR0z4cPXkD1,"=image(""https://drive.google.com/uc?export=dow..."
4,BID,VNINDEX,BIDV,HOSE,Ngân hàng,1993,http://www.bidv.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",19Up94QCdCDvoJURylxYzQKEGECs81i0F,"=image(""https://drive.google.com/uc?export=dow..."
5,BWE,VNINDEX,Nước - Môi trường Bình Dương,HOSE,"Điện, nước & xăng dầu khí đốt",2006,http://www.biwase.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",1s6NaEHY51PBSOCtZ_O9YzS74YcEeG8JF,"=image(""https://drive.google.com/uc?export=dow..."
6,BVH,VNINDEX,Tập đoàn Bảo Việt,HOSE,Bảo hiểm,2007,https://www.baoviet.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",1u-Q-VhUICrxxMxhQefqSSt6wGH1rxNhc,"=image(""https://drive.google.com/uc?export=dow..."
7,CRE,VNINDEX,Bất động sản Thế Kỷ,HOSE,Bất động sản,2001,http://www.cenland.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",1Q9KOaUlnWvezjFZBFhqtyL84MraRwoJE,"=image(""https://drive.google.com/uc?export=dow..."
8,CII,VNINDEX,Hạ tầng Kỹ thuật TP.HCM,HOSE,Xây dựng và Vật liệu,2001,http://www.cii.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",1mJFGAMPQqexSixujcKBb7P9SDeRnsiVV,"=image(""https://drive.google.com/uc?export=dow..."
9,CMG,VNINDEX,Tập đoàn Công nghệ CMC,HOSE,Công nghệ Thông tin,2009,http://www.cmc.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...",1LXvXiCRNZgtO70O7Ewl4fONS9karjh0w,"=image(""https://drive.google.com/uc?export=dow..."


In [12]:
df_merge['3 years chart'][0]

'=image("https://drive.google.com/uc?export=download&id=1fVtzCw4Wyg1k9VJuocSfPhsJ3-oC8SjH")'

# Share Anyone

In [13]:
'''
request_body = {'role': 'reader',
                'type': 'anyone'}

for file_id in list_id:
    response_permission = service.permissions().create(fileId=file_id, body=request_body).execute()
    print(file_id)
'''

"\nrequest_body = {'role': 'reader',\n                'type': 'anyone'}\n\nfor file_id in list_id:\n    response_permission = service.permissions().create(fileId=file_id, body=request_body).execute()\n    print(file_id)\n"

# Replace Chart trên Drive

In [14]:
df_merge['id'][0]

'1fVtzCw4Wyg1k9VJuocSfPhsJ3-oC8SjH'

In [15]:
df_merge['ticker'][0]

'FUEIP100'

In [16]:
len(df_merge)

45

In [17]:
for i in range(len(df_merge)):
    file_id = df_merge['id'][i]
    media_content = MediaFileUpload(fr"E:\OneDrive\CNTT\Project Python\Python Finance\Chart\\{df_merge['ticker'][i]}.jpg", mimetype='image/png') #thay thế bằng file và mimeType này
    #media_content = MediaFileUpload(fr"E:\Chart\\{df_merge['ticker'][i]}.jpg", mimetype='image/png')
    response = service.files().update(fileId=file_id, media_body=media_content).execute()
    print(response)

{'kind': 'drive#file', 'id': '1fVtzCw4Wyg1k9VJuocSfPhsJ3-oC8SjH', 'name': 'FUEIP100.jpg', 'mimeType': 'image/jpeg'}
{'kind': 'drive#file', 'id': '15x07ENc_DvTIg0yIW0s30ydbM1r63Gde', 'name': 'FUEKIV30.jpg', 'mimeType': 'image/jpeg'}
{'kind': 'drive#file', 'id': '1OBE7VAKNFNMWWL9M_aBiws1DnkQSJ1iq', 'name': 'AGG.jpg', 'mimeType': 'image/jpeg'}
{'kind': 'drive#file', 'id': '1KoSM2mz6y2jiT0ndFFEtYpR0z4cPXkD1', 'name': 'ANV.jpg', 'mimeType': 'image/jpeg'}
{'kind': 'drive#file', 'id': '19Up94QCdCDvoJURylxYzQKEGECs81i0F', 'name': 'BID.jpg', 'mimeType': 'image/jpeg'}
{'kind': 'drive#file', 'id': '1s6NaEHY51PBSOCtZ_O9YzS74YcEeG8JF', 'name': 'BWE.jpg', 'mimeType': 'image/jpeg'}
{'kind': 'drive#file', 'id': '1u-Q-VhUICrxxMxhQefqSSt6wGH1rxNhc', 'name': 'BVH.jpg', 'mimeType': 'image/jpeg'}
{'kind': 'drive#file', 'id': '1Q9KOaUlnWvezjFZBFhqtyL84MraRwoJE', 'name': 'CRE.jpg', 'mimeType': 'image/jpeg'}
{'kind': 'drive#file', 'id': '1mJFGAMPQqexSixujcKBb7P9SDeRnsiVV', 'name': 'CII.jpg', 'mimeType': 'imag

# Final DataFrame

In [18]:
df_final = pd.merge(df_merge, change, left_on='ticker', right_on='Symbol')

df_final = df_final[['ticker', 'company_short_name', 'industry', 'website', 'Link Chart', '3 years chart',
                    'Price', 'Max 3 Years', 'Good Price', '1 day', '30days', '3years']]
df_final['Good Price'] = df_final['Good Price'].map({True: 'YES', False: 'NO'})
df_final.replace([np.inf, -np.inf], 0, inplace=True)
df_final

,ticker,company_short_name,industry,website,Link Chart,3 years chart,Price,Max 3 Years,Good Price,1 day,30days,3years
0,FUEIP100,ETF IPAAM VN100,Dịch vụ tài chính,https://ipaam.com.vn/vi/home,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",7840.0,0.0,NO,0.38,0,0
1,FUEKIV30,KIM GROWTH VN30 ETF,Dịch vụ tài chính,https://koreainvestment.com.vn/en/home,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",7280.0,0.0,NO,0.55,0,0
2,AGG,Bất động sản An Gia,Bất động sản,https://angia.com.vn/,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",29850.0,0.0,NO,0.00,0,0
3,ANV,Thủy sản Nam Việt,Thực phẩm và đồ uống,http://www.navicorp.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",31100.0,61560.0,NO,0.32,0,0
4,BID,BIDV,Ngân hàng,http://www.bidv.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",45950.0,49000.0,NO,1.88,0,0
5,BWE,Nước - Môi trường Bình Dương,"Điện, nước & xăng dầu khí đốt",http://www.biwase.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",45500.0,61500.0,NO,-0.55,0,0
6,BVH,Tập đoàn Bảo Việt,Bảo hiểm,https://www.baoviet.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",49900.0,88647.0,NO,3.10,0,0
7,CRE,Bất động sản Thế Kỷ,Bất động sản,http://www.cenland.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",8560.0,25621.0,YES,-0.70,0,0
8,CII,Hạ tầng Kỹ thuật TP.HCM,Xây dựng và Vật liệu,http://www.cii.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",14650.0,57900.0,YES,-1.01,0,0
9,CMG,Tập đoàn Công nghệ CMC,Công nghệ Thông tin,http://www.cmc.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",41600.0,53000.0,NO,1.46,0,0


# Up lên Google Sheet

In [19]:
#Dịch vụ cần chạy
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(r'E:\OneDrive\CNTT\Project Python\Python Finance\credential_GGSheet_wenlamsao.json',scopes=scopes)
#credentials = Credentials.from_service_account_file(r'E:\credential_GGSheet_wenlamsao.json',scopes=scopes)
#Start job
gc = gspread.authorize(credentials) #xác nhận để mở khoá Python với Google Sheet

In [20]:
# Get destination file
#sh = gc.open_by_key('15IL_zoIJwpFV0Zztad6NPbCQwi4LBL7DdpkLFqcjq4g').worksheet("Sheet1")
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/15IL_zoIJwpFV0Zztad6NPbCQwi4LBL7DdpkLFqcjq4g').worksheet("VN100")
sh.clear() #Clear Data

#Update data
a = sh.update([df_final.columns.values.tolist()] + df_final.values.tolist(), raw=False)
A1 = sh.update('A1', f'{str_today}')
B1 = sh.update('B1', 'Name')
C1 = sh.update('C1', 'Industry')
D1 = sh.update('D1', 'Website')

print('-------------------------- ALL THING DONE --------------------------')

-------------------------- ALL THING DONE --------------------------


# Send Messages Telegram

In [21]:
df_final['Messages Telegram'] = df_final['ticker'] + ' - ' + df_final['company_short_name'] + ' - ' + df_final['Good Price']
print(list(df_final['Messages Telegram']))
df_final

['FUEIP100 - ETF IPAAM VN100 - NO', 'FUEKIV30 - KIM GROWTH VN30 ETF - NO', 'AGG - Bất động sản An Gia - NO', 'ANV - Thủy sản Nam Việt - NO', 'BID - BIDV - NO', 'BWE - Nước - Môi trường Bình Dương - NO', 'BVH - Tập đoàn Bảo Việt - NO', 'CRE - Bất động sản Thế Kỷ - YES', 'CII - Hạ tầng Kỹ thuật TP.HCM - YES', 'CMG - Tập đoàn Công nghệ CMC - NO', 'CTD - Xây dựng Coteccons - YES', 'CTG - VietinBank - NO', 'DGC - Hóa chất Đức Giang - YES', 'DIG - DIC Corp - YES', 'DXG - Địa ốc Đất Xanh - YES', 'DXS - Dịch vụ BĐS Đất Xanh - NO', 'EIB - Eximbank - NO', 'FPT - FPT Corp - NO', 'FUESSV30 - Quỹ ETF SSIAM VN30 - NO', 'FUEVN100 - Quỹ ETF VINACAPITAL VN100 - NO', 'GEX - Tập đoàn Gelex - YES', 'GEG - Điện Gia Lai - NO', 'HDB - HDBank - NO', 'HDG - Tập đoàn Hà Đô - NO', 'TCH - ĐT DV Tài chính Hoàng Huy - YES', 'HNG - Nông nghiệp Quốc tế HAGL - YES', 'ITA - Đầu tư CN Tân Tạo - YES', 'KBC - TCT Đô thị Kinh Bắc - NO', 'KDC - Tập đoàn KIDO - NO', 'LPB - LienViet Post Bank - NO', 'MSN - Tập đoàn Masan - N

,ticker,company_short_name,industry,website,Link Chart,3 years chart,Price,Max 3 Years,Good Price,1 day,30days,3years,Messages Telegram
0,FUEIP100,ETF IPAAM VN100,Dịch vụ tài chính,https://ipaam.com.vn/vi/home,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",7840.0,0.0,NO,0.38,0,0,FUEIP100 - ETF IPAAM VN100 - NO
1,FUEKIV30,KIM GROWTH VN30 ETF,Dịch vụ tài chính,https://koreainvestment.com.vn/en/home,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",7280.0,0.0,NO,0.55,0,0,FUEKIV30 - KIM GROWTH VN30 ETF - NO
2,AGG,Bất động sản An Gia,Bất động sản,https://angia.com.vn/,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",29850.0,0.0,NO,0.00,0,0,AGG - Bất động sản An Gia - NO
3,ANV,Thủy sản Nam Việt,Thực phẩm và đồ uống,http://www.navicorp.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",31100.0,61560.0,NO,0.32,0,0,ANV - Thủy sản Nam Việt - NO
4,BID,BIDV,Ngân hàng,http://www.bidv.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",45950.0,49000.0,NO,1.88,0,0,BID - BIDV - NO
5,BWE,Nước - Môi trường Bình Dương,"Điện, nước & xăng dầu khí đốt",http://www.biwase.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",45500.0,61500.0,NO,-0.55,0,0,BWE - Nước - Môi trường Bình Dương - NO
6,BVH,Tập đoàn Bảo Việt,Bảo hiểm,https://www.baoviet.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",49900.0,88647.0,NO,3.10,0,0,BVH - Tập đoàn Bảo Việt - NO
7,CRE,Bất động sản Thế Kỷ,Bất động sản,http://www.cenland.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",8560.0,25621.0,YES,-0.70,0,0,CRE - Bất động sản Thế Kỷ - YES
8,CII,Hạ tầng Kỹ thuật TP.HCM,Xây dựng và Vật liệu,http://www.cii.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",14650.0,57900.0,YES,-1.01,0,0,CII - Hạ tầng Kỹ thuật TP.HCM - YES
9,CMG,Tập đoàn Công nghệ CMC,Công nghệ Thông tin,http://www.cmc.com.vn,"=HYPERLINK(""https://liveboard.cafef.vn/ptkt?sy...","=image(""https://drive.google.com/uc?export=dow...",41600.0,53000.0,NO,1.46,0,0,CMG - Tập đoàn Công nghệ CMC - NO


In [22]:
base_url = "https://api.telegram.org/bot5968317951:AAG8KN0Wyc2EjqtGKuoIcZOsrAMeqy-0PYw/sendMessage"
texts = list(df_final['Messages Telegram'])
for text in texts:
    sleep(1)
    parameters = {"chat_id": "-898774994", 
                "text": text} 
    resp = requests.get(base_url, data=parameters)
    #print(resp.text) 

# Xử lý Danh mục đầu tư

### Đọc dữ liệu Google Sheet Danh Mục Đầu tư chứng khoán

In [31]:
#Dịch vụ cần chạy
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(r'E:\OneDrive\CNTT\MindX - 4 - Data Analyst Career Kickstart\Python voi BigQuery\credential_GGSheet_wenlamsao.json',scopes=scopes)
gc = pygsheets.authorize(custom_credentials=credentials)

#sh = gc.open_by_key('spreadsheet_key')
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kSqlqo05pWd1ASn9gn03-0O-DwCbx1Cy3b_DLmeXPmI/edit#gid=841255028').worksheet_by_title("Đầu tư chứng khoán")
sh

<Worksheet 'Đầu tư chứng khoán' index:1>

In [32]:
df_ggsheet = sh.get_as_df() #you can also get the values of sheet as dataframe
df_ggsheet['Giá mua'] = df_ggsheet['Giá mua'].str.replace(',','.', regex=True).astype(float)
df_ggsheet

df_ggsheet['Số tiền mua'] = df_ggsheet['Số tiền mua']*1000
df_ggsheet[['Ngày mua']] = df_ggsheet[['Ngày mua']].apply(pd.to_datetime, dayfirst=True)

def calculate_months(date):
    now = datetime.now() 
    diff = now - date 
    return round(diff / np.timedelta64(1,'D'))
df_ggsheet['Số ngày'] = df_ggsheet['Ngày mua'].apply(calculate_months).tolist()
df_ggsheet

,Symbol,Giá mua,Số tiền mua,Ngày mua,Số ngày,Giá hiện tại,Lãi / Lỗ,% Lãi / Lỗ
0,HPG,60.5,1000.0,2023-01-10,14,21,-650,"-0,65"
1,VIC,50.1,1000.0,2023-01-15,9,57,150,"0,15"


### Đưa giá hiện tại từ DataFrame Change vào

In [33]:
df_change = change.copy()
df_change['Price'] = df_change['Price']/1000
#df_change['Price'] = df_change['Price'].astype(int)
df_change.sort_values(by='Symbol',ascending=True,inplace=True) #bỏ inplace nếu ko cần gắn trực tiếp vào DataFrame
df_change

,Symbol,Price,Max 3 Years,Good Price,1 day,30days,3years
0,AAA,7.59,22800.0,True,0.13,0,0
1,ACB,26.00,30360.0,False,1.96,0,0
2,AGG,29.85,0.0,False,0.00,0,0
3,ANV,31.10,61560.0,False,0.32,0,0
4,APH,7.36,0.0,False,0.27,0,0
...,...,...,...,...,...,...,...
93,VNM,81.30,106304.0,False,0.00,0,0
94,VPB,19.50,26929.0,False,-0.76,0,0
95,VPI,53.00,61000.0,False,0.19,0,0
96,VRE,29.65,37800.0,False,1.19,0,0


In [34]:
price_hientai = df_change[df_change['Symbol'].isin(df_ggsheet['Symbol'])]['Price'] # lọc theo danh sách list
df_ggsheet['Giá hiện tại'] = list(price_hientai)

df_ggsheet['% Lãi / Lỗ'] = ((df_ggsheet['Giá hiện tại'] - df_ggsheet['Giá mua']) / df_ggsheet['Giá mua']).round(2)
df_ggsheet['Lãi / Lỗ'] = (df_ggsheet['Số tiền mua'] * df_ggsheet['% Lãi / Lỗ']).round(2)
df_ggsheet

,Symbol,Giá mua,Số tiền mua,Ngày mua,Số ngày,Giá hiện tại,Lãi / Lỗ,% Lãi / Lỗ
0,HPG,60.5,1000.0,2023-01-10,14,21.15,-650.0,-0.65
1,VIC,50.1,1000.0,2023-01-15,9,57.40,150.0,0.15


### Up lên lại Google Sheet

In [35]:
#chuyển lại thành string để up GGSheet
df_ggsheet_new = df_ggsheet.copy()
df_ggsheet_new['Giá mua'] = df_ggsheet_new['Giá mua'].astype('string').str.replace('.',',', regex=True) 
df_ggsheet_new['Số tiền mua'] = df_ggsheet_new['Số tiền mua'].astype('string').str.replace('.',',', regex=True) 
df_ggsheet_new['Giá hiện tại'] = df_ggsheet_new['Giá hiện tại'].astype('string').str.replace('.',',', regex=True) 
df_ggsheet_new['Lãi / Lỗ'] = df_ggsheet_new['Lãi / Lỗ'].astype('string').str.replace('.',',', regex=True) 
df_ggsheet_new['% Lãi / Lỗ'] = df_ggsheet_new['% Lãi / Lỗ'].astype('string').str.replace('.',',', regex=True)
df_ggsheet_new

,Symbol,Giá mua,Số tiền mua,Ngày mua,Số ngày,Giá hiện tại,Lãi / Lỗ,% Lãi / Lỗ
0,HPG,"60,5","1000,0",2023-01-10,14,"21,15","-650,0","-0,65"
1,VIC,"50,1","1000,0",2023-01-15,9,"57,4","150,0","0,15"


In [36]:
sh.clear()
sh.set_dataframe(df_ggsheet_new, 'A1', raw=False) #copy_head=False if you don't want to add first row of df